In [1]:
import os, sys
sys.path.insert(1, 'D:\Github\DeepLake-Langchain')
import credentials
os.environ["OPENAI_API_KEY"] = credentials.openai
os.environ["ACTIVELOOP_TOKEN"] =credentials.active_loop

In [2]:
# We scrape several Artificial Intelligence news

import requests
from newspaper import Article # https://github.com/codelucas/newspaper
import time

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/05/23/meta-open-source-speech-ai-models-support-over-1100-languages/",
    "https://www.artificialintelligence-news.com/2023/05/18/beijing-launches-campaign-against-ai-generated-misinformation/"
    "https://www.artificialintelligence-news.com/2023/05/16/openai-ceo-ai-regulation-is-essential/",
    "https://www.artificialintelligence-news.com/2023/05/15/jay-migliaccio-ibm-watson-on-leveraging-ai-to-improve-productivity/",
    "https://www.artificialintelligence-news.com/2023/05/15/iurii-milovanov-softserve-how-ai-ml-is-helping-boost-innovation-and-personalisation/",
    "https://www.artificialintelligence-news.com/2023/05/11/ai-and-big-data-expo-north-america-begins-in-less-than-one-week/",
    "https://www.artificialintelligence-news.com/2023/05/11/eu-committees-green-light-ai-act/",
    "https://www.artificialintelligence-news.com/2023/05/09/wozniak-warns-ai-will-power-next-gen-scams/",
    "https://www.artificialintelligence-news.com/2023/05/09/infocepts-ceo-shashank-garg-on-the-da-market-shifts-and-impact-of-ai-on-data-analytics/",
    "https://www.artificialintelligence-news.com/2023/05/02/ai-godfather-warns-dangers-and-quits-google/",
    "https://www.artificialintelligence-news.com/2023/04/28/palantir-demos-how-ai-can-used-military/",
    "https://www.artificialintelligence-news.com/2023/04/26/ftc-chairwoman-no-ai-exemption-to-existing-laws/",
    "https://www.artificialintelligence-news.com/2023/04/24/bill-gates-ai-teaching-kids-literacy-within-18-months/",
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/"
]

session = requests.Session()
pages_content = [] # where we save the scraped articles

for url in article_urls:
    try:
        time.sleep(2) # sleep two seconds for gentle scraping
        response = session.get(url, headers=headers, timeout=10)

        if response.status_code == 200:
            article = Article(url)
            article.download() # download HTML of webpage
            article.parse() # parse HTML to extract the article text
            pages_content.append({ "url": url, "text": article.text })
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

#If an error occurs while fetching an article, we catch the exception and print
#an error message. This ensures that even if one article fails to download,
#the rest of the articles can still be processed.

In [3]:
# We'll use an embedding model to compute our documents' embeddings
from langchain.embeddings.openai import OpenAIEmbeddings

# We'll store the documents and their embeddings in the deep lake vector db
from langchain.vectorstores import DeepLake

# Setup deep lake
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# create Deep Lake dataset
my_activeloop_org_id = credentials.active_loop_org_id
my_activeloop_dataset_name = "analysis_agent"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

d:\Anaconda\envs\deeplakelangchain\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.11) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


Your Deep Lake dataset has been successfully created!
The dataset is private so make sure you are logged in!


\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/megatron17/analysis_agent


hub://megatron17/analysis_agent loaded successfully.


In [4]:
# We split the article texts into small chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

all_texts = []
for d in pages_content:
    chunks = text_splitter.split_text(d["text"])
    for chunk in chunks:
        all_texts.append(chunk)

In [5]:
# we add all the chunks to the Deep lake
db.add_texts(all_texts)

Evaluating ingest: 0%|          | 0/1 [00:00<?Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your 

Dataset(path='hub://megatron17/analysis_agent', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype     shape      dtype  compression
  -------   -------   -------    -------  ------- 
 embedding  generic  (94, 1536)  float32   None   
    ids      text     (94, 1)      str     None   
 metadata    json     (94, 1)      str     None   
   text      text     (94, 1)      str     None   


['9b75246f-26e0-11ee-9785-00d861dd19c7',
 '9b752470-26e0-11ee-af1e-00d861dd19c7',
 '9b752471-26e0-11ee-9329-00d861dd19c7',
 '9b752472-26e0-11ee-9c18-00d861dd19c7',
 '9b752473-26e0-11ee-aaea-00d861dd19c7',
 '9b752474-26e0-11ee-9f6a-00d861dd19c7',
 '9b752475-26e0-11ee-89df-00d861dd19c7',
 '9b752476-26e0-11ee-aab7-00d861dd19c7',
 '9b752477-26e0-11ee-ac6a-00d861dd19c7',
 '9b752478-26e0-11ee-a195-00d861dd19c7',
 '9b752479-26e0-11ee-8ffa-00d861dd19c7',
 '9b75247a-26e0-11ee-af8b-00d861dd19c7',
 '9b75247b-26e0-11ee-8fc9-00d861dd19c7',
 '9b75247c-26e0-11ee-8ef3-00d861dd19c7',
 '9b75247d-26e0-11ee-bd9b-00d861dd19c7',
 '9b75247e-26e0-11ee-997d-00d861dd19c7',
 '9b75247f-26e0-11ee-bbb6-00d861dd19c7',
 '9b752480-26e0-11ee-8562-00d861dd19c7',
 '9b752481-26e0-11ee-a15e-00d861dd19c7',
 '9b752482-26e0-11ee-82d6-00d861dd19c7',
 '9b752483-26e0-11ee-8dc5-00d861dd19c7',
 '9b752484-26e0-11ee-8eea-00d861dd19c7',
 '9b752485-26e0-11ee-997e-00d861dd19c7',
 '9b752486-26e0-11ee-820c-00d861dd19c7',
 '9b752487-26e0-

In [6]:
# Get the retriever object from the deep lake db object and set the number
# of retrieved documents to 3
retriever = db.as_retriever()
retriever.search_kwargs['k'] = 3

# We define some variables that will be used inside our custom tool
CUSTOM_TOOL_DOCS_SEPARATOR ="\n---------------\n" # how to join together the retrieved docs to form a single string

# This is the function that defines our custom tool that retrieves relevant
# docs from Deep Lake
def retrieve_n_docs_tool(query: str) -> str:
    """Searches for relevant documents that may contain the answer to the query."""
    docs = retriever.get_relevant_documents(query)
    texts = [doc.page_content for doc in docs]
    texts_merged = "---------------\n" + CUSTOM_TOOL_DOCS_SEPARATOR.join(texts) + "\n---------------"
    return texts_merged

In [7]:
from langchain.agents.tools import Tool

# We create the tool that uses the "retrieve_n_docs_tool" function
tools = [
    Tool(
        name="Search Private Docs",
        func=retrieve_n_docs_tool,
        description="useful for when you need to answer questions about current events about Artificial Intelligence"
    )
]

In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.experimental.plan_and_execute import PlanAndExecute, load_agent_executor, load_chat_planner

# let's create the Plan and Execute agent
model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
planner = load_chat_planner(model)
executor = load_agent_executor(model, tools, verbose=True)
agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [9]:
# we test the agent
response = agent.run("Write an overview of Artificial Intelligence regulations by governments by country")



> Entering new  chain...
steps=[Step(value='Research and gather information on the current state of Artificial Intelligence (AI) regulations by governments in different countries.'), Step(value='Organize the information by country and categorize it based on the level of AI regulation.'), Step(value='Provide an overview of the AI regulations in each country, including any specific laws or policies that have been implemented.'), Step(value='Include information on the key areas covered by the regulations, such as data privacy, algorithmic transparency, liability, and ethical considerations.'), Step(value='Highlight any notable differences or similarities between the regulations in different countries.'), Step(value='Summarize the overall trends and developments in AI regulations globally.'), Step(value='Given the above steps taken, provide an overview of Artificial Intelligence regulations by governments by country. ')]

> Entering new  chain...
Action:
```
{
  "action": "Search Private

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Action:
```
{
  "action": "Search Private Docs",
  "action_input": "AI regulations overview by country"
}
```
Observation: ---------------
“US-based AI developers will likely steal a march on their European competitors given the news that the EU parliamentary committees have green-lit its groundbreaking AI Act, where AI systems will need to be categorised according to their potential for harm from the outset. The US tech approach is typically to experiment first and, once market and product fit is established, to retrofit to other markets and their regulatory framework. This approach fosters innovation whereas EU-based AI developers will need to take note of the new rules and develop systems and processes which may take the edge off their ability to innovate. The UK is adopting a similar approach to the US, although the proximity of the EU market means that UK-based developers are more likely to fall into step with the EU ruleset from the outset. However, the potential to experiment in

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Based on the previous steps and the information provided, it seems that the AI regulations overview by country has already been researched and organized. The information includes the current state of AI regulations in different countries, such as the European Union, United States, United Kingdom, and China. The EU has recently approved the AI Act, categorizing AI systems based on their potential for harm. The US approach is more focused on experimentation and market fit, with regulations in place for specific sectors. The UK is likely to align with the EU ruleset, and China has implemented AI regulations addressing data privacy and security concerns.

Since the objective is to provide an overview of AI regulations in each country, including specific laws or policies, it seems that the previous research and organization already covers this. Therefore, the next step would be to provide the overview to the user.

Action:
```
{
  "action": "Final Answer",
  "action_input": "The overview of

...

Based on the previous steps, the information on the key areas covered by AI regulations in different countries was not provided. To include this information, I can search for it in the private documents.

Action:
```
{
  "action": "Search Private Docs",
  "action_input": "AI regulations key areas covered"
}
```

Observation: ---------------
In a demo showcasing AIP, a military operator responsible for monitoring activity within Eastern Europe receives an alert that military equipment is amassed in a field 30km from friendly forces.

AIP leverages large language models to allow operators to quickly ask questions such as:

What enemy units are in the region?

Task new imagery for this location at a resolution of one metre or higher

Generate three courses of action to target this enemy equipment

Analyse the battlefield, considering a Stryker vehicle and a platoon-size unit

How many Javelin missiles does Team Omega have?

Assign jammers to each of the validated high-priority communicati

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

Based on the previous steps, the information on the key areas covered by AI regulations in different countries was not provided. To include this information, I can search for it in the private documents.

Action:
```
{
  "action": "Search Private Docs",
  "action_input": "AI regulations key areas covered"
}
```


Observation: ---------------
In a demo showcasing AIP, a military operator responsible for monitoring activity within Eastern Europe receives an alert that military equipment is amassed in a field 30km from friendly forces.

AIP leverages large language models to allow operators to quickly ask questions such as:

What enemy units are in the region?

Task new imagery for this location at a resolution of one metre or higher

Generate three courses of action to target this enemy equipment

Analyse the battlefield, considering a Stryker vehicle and a platoon-size unit

How many Javelin missiles does Team Omega have?

Assign jammers to each of the validated high-priority communicat

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-gHQJ1WmbtrejM3Byo6KJyQPD on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 4385 tokens. Please reduce the length of the messages.